In [1]:
import pandas as pd

# load the necessary Pulp function
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

In [13]:

# read data from excel file 'project_plan.xlsx'
# file is stored at '002. Input Tables'

input_file_path = '002. Input Tables/project_plan.xlsx'



# read data from excel file 'project_plan_simple.xlsx'
# if NaN then replace by None,
# if column is string then remove trailing characters
def read_data(file_path):
    df = pd.read_excel(file_path, keep_default_na=False)
    return df

# read data from excel file 'project_plan_simple.xlsx'
df = read_data(input_file_path)

# display data as markdown table
print(df.to_markdown())

# export markdown table to '003. Output Files/table_01_project_plan.md'
output_file_path = '003. Output Files/table_01_project_plan.md'
with open(output_file_path, 'w') as f:
    f.write(df.to_markdown())

|    | taskID   | task                         | predecessorTaskIDs     |   bestCaseHours | expectedHours   |   worstCaseHours | projectManager   | frontendDeveloper   | backendDeveloper   | dataScientist   | dataEngineer   |   Rate_Hour |   Quantity_People |   Total_Costs |   PM_Costs |   Dev_Team_Costs |   Maximum Reduction |   Crashing |   Crash_Cost_Hour |
|---:|:---------|:-----------------------------|:-----------------------|----------------:|:----------------|-----------------:|:-----------------|:--------------------|:-------------------|:----------------|:---------------|------------:|------------------:|--------------:|-----------:|-----------------:|--------------------:|-----------:|------------------:|
|  0 | A        | Describe product             |                        |           20    | 20              |             20   | 1                |                     |                    |                 |                |          60 |                 1 |          1200 

In [ ]:
# more trailing characters from all columns in the dataframe
# apply lambda if x is string, then remove trailing characters
def remove_trailing_characters(df):
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    return df

# remove trailing characters from all columns in the dataframe
df = remove_trailing_characters(df)

# remove ',' from predecessorTaskIDs and add element to a list
# removving trailing spaces
# using apply() + lambda
df['predecessorTaskIDs'] = df['predecessorTaskIDs'].apply(lambda x: x.split(',')).tolist()

# remove trailing spaces
df['predecessorTaskIDs'] = df['predecessorTaskIDs'].apply(lambda x: [i.strip() for i in x]).tolist()

In [62]:
# create dictionary with data
# taskID and task
taskID_task = dict(zip(df['taskID'], df['task']))
activities_list = taskID_task.keys()

# create a dictionary with data
# taskID and expected_duration
taskID_expectedHours = dict(zip(df['taskID'], df['expectedHours']))
activities = taskID_expectedHours

# create a list to the element of the column predecessorTaskIDs
predecessorTaskIDs = df['predecessorTaskIDs'].tolist()

# create a dictionary with data
# taskID and predecessorTaskIDs
taskID_predecessorTaskIDs = dict(zip(df['taskID'], df['predecessorTaskIDs']))
precedences = taskID_predecessorTaskIDs

In [63]:
# print activities_list,activities,precedences
print(activities_list)
print(activities)
print(precedences)


dict_keys(['A', 'B', 'C', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'E', 'F', 'G', 'H'])
{'A': 20, 'B': 40, 'C': 20, 'D1': 40, 'D2': 30, 'D3': 30, 'D4': 150, 'D5': 20, 'D6': 40, 'D7': 40, 'D8': 20, 'E': 30, 'F': 15, 'G': 30, 'H': 15}
{'A': [''], 'B': [''], 'C': ['A'], 'D1': ['A'], 'D2': ['D1'], 'D3': ['D1'], 'D4': ['D2', 'D3'], 'D5': ['D4'], 'D6': ['D4'], 'D7': ['D6'], 'D8': ['D5', 'D7'], 'E': ['B', 'C'], 'F': ['D8', 'E'], 'G': ['A', 'D8'], 'H': ['F', 'G']}


In [64]:
# for all the values in the precendences dictionary
# that are '' replace by empty list
for key, value in precedences.items():
    if value == ['']:
        precedences[key] = []

# print precedences
print(precedences)

{'A': [], 'B': [], 'C': ['A'], 'D1': ['A'], 'D2': ['D1'], 'D3': ['D1'], 'D4': ['D2', 'D3'], 'D5': ['D4'], 'D6': ['D4'], 'D7': ['D6'], 'D8': ['D5', 'D7'], 'E': ['B', 'C'], 'F': ['D8', 'E'], 'G': ['A', 'D8'], 'H': ['F', 'G']}


In [65]:
# Critical Path Analysis 

# Problem description from Williams (2013, pages 94-98)
# Williams, H. Paul. 2013. Model Building in Mathematical Programming (fifth edition). New York: Wiley. [ISBN-13: 978-1-118-44333-0]

# Python PuLP solution prepared by Thomas W. Miller
# Revised April 20, 2023
# Implemented using activities dictionary with derived start_times and end_times
# rather than time decision variables as in Williams (2013)

from pulp import *

# Create a dictionary of the activities and their durations
#activities = {'DigFoundations':4, 'LayFoundations':2, 'ObtainBricks':7, 'ObtainTiles':12, 'Walls':10, 'Roofing':5, 'Wiring':3, 'Painting':4}

# Create a list of the activities
#activities_list = list(activities.keys())

# Create a dictionary of the activity precedences
#precedences = {'DigFoundations': [], 'ObtainBricks': [], 'ObtainTiles': [], 'LayFoundations': ['DigFoundations'],  'Walls': ['DigFoundations','ObtainBricks'], 'Wiring': ['Walls'], 'Roofing': ['ObtainTiles','Walls'], 'Painting': ['Wiring','Roofing']}



In [66]:
# Create the LP problem
prob = LpProblem("Critical_Path", LpMinimize)

# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

In [67]:
value(start_times['A'])

In [68]:
# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        if predecessor != []:
            prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if (start_times[activity].value()) == 0:
        print(f"{activity} starts at time 0")
    if (end_times[activity].value()) == max([end_times[activity].value() for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} hours in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)


Critical Path time:
A starts at time 0
B starts at time 0
H ends at 385.0 hours in duration

Solution variable values:
end_A = 20.0
end_B = 40.0
end_C = 40.0
end_D1 = 60.0
end_D2 = 90.0
end_D3 = 90.0
end_D4 = 240.0
end_D5 = 260.0
end_D6 = 280.0
end_D7 = 320.0
end_D8 = 340.0
end_E = 70.0
end_F = 355.0
end_G = 370.0
end_H = 385.0
start_A = 0.0
start_B = 0.0
start_C = 20.0
start_D1 = 20.0
start_D2 = 60.0
start_D3 = 60.0
start_D4 = 90.0
start_D5 = 240.0
start_D6 = 240.0
start_D7 = 280.0
start_D8 = 320.0
start_E = 40.0
start_F = 340.0
start_G = 340.0
start_H = 370.0


In [69]:
import pandas as pd

# display the shadow prices, and slack of the constraints
o = [(name, c.pi, c.slack) for name, c in prob.constraints.items()]

df_1 = pd.DataFrame(o, columns=['name', 'shadow price', 'slack'])

# print df without the index and to markdown
print(df_1.to_markdown(index=False))

# display the reduced costs of the variables
o = [(v.name, v.dj) for v in prob.variables()]

df_2 = pd.DataFrame(o, columns=['name', 'reduced cost'])

# print df_2 without the index and to markdown
print(df_2.to_markdown(index=False))


| name              |   shadow price |   slack |
|:------------------|---------------:|--------:|
| A_duration        |             13 |      -0 |
| B_duration        |              2 |      -0 |
| C_duration        |              1 |      -0 |
| C_predecessor_A   |              1 |      -0 |
| D1_duration       |             11 |      -0 |
| D1_predecessor_A  |             11 |      -0 |
| D2_duration       |              1 |      -0 |
| D2_predecessor_D1 |              1 |      -0 |
| D3_duration       |              9 |      -0 |
| D3_predecessor_D1 |              9 |      -0 |
| D4_duration       |              8 |      -0 |
| D4_predecessor_D2 |              0 |      -0 |
| D4_predecessor_D3 |              8 |      -0 |
| D5_duration       |              1 |      -0 |
| D5_predecessor_D4 |              1 |      -0 |
| D6_duration       |              6 |      -0 |
| D6_predecessor_D4 |              6 |      -0 |
| D7_duration       |              5 |      -0 |
| D7_predecessor_D6 

In [70]:
# Use GLPK for sensitivity analysis
prob.writeLP("critical_path.lp")
prob.solve(GLPK(options=['--ranges critical_path.sen']))
print ("Status:", LpStatus[prob.status])



PulpSolverError: PuLP: cannot execute glpsol.exe